Cada línea del archivo movies.csv representa a un actor que actuó en una película. Si una película tiene diez actores, habrá diez filas para esa película en particular.



1. Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

2. Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.

3. Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

In [13]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, count, desc, udf, rank, countDistinct, when, lit, row_number, rank, coalesce, dense_rank, max
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [3]:
movies_df = spark.read.csv("../data/s9_in/movies.csv", header=True, inferSchema=True, sep="|")
movie_ratings_df = spark.read.csv("../data/s9_in/movie_ratings.csv", header=True, inferSchema=True, sep="|")

movies_df.printSchema()
movie_ratings_df.printSchema()

movies_df.show(5, truncate=False)
movie_ratings_df.show(5, truncate=False)

root
 |-- actor: string (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: integer (nullable = true)

root
 |-- valoracion: double (nullable = true)
 |-- pelicula: string (nullable = true)
 |-- año: integer (nullable = true)

+-----------------+-------------+----+
|actor            |pelicula     |año |
+-----------------+-------------+----+
|McClure, Marc (I)|Freaky Friday|2003|
|McClure, Marc (I)|Coach Carter |2005|
|McClure, Marc (I)|Superman II  |1980|
|McClure, Marc (I)|Apollo 13    |1995|
|McClure, Marc (I)|Superman     |1978|
+-----------------+-------------+----+
only showing top 5 rows

+----------+--------------------------+----+
|valoracion|pelicula                  |año |
+----------+--------------------------+----+
|1.6339    |'Crocodile' Dundee II     |1988|
|7.6177    |10                        |1979|
|1.2864    |10 Things I Hate About You|1999|
|0.3243    |10,000 BC                 |2008|
|0.3376    |101 Dalmatians            |1996|
+----------+---------

In [4]:
# 1
movies_df.groupBy("actor").count().orderBy(desc("count")).show()

+-------------------+-----+
|              actor|count|
+-------------------+-----+
|   Tatasciore, Fred|   38|
|      Welker, Frank|   38|
| Jackson, Samuel L.|   32|
|      Harnell, Jess|   31|
|        Damon, Matt|   27|
|      Willis, Bruce|   27|
|  Cummings, Jim (I)|   26|
|         Hanks, Tom|   25|
|   Lynn, Sherry (I)|   25|
|    Bergen, Bob (I)|   25|
|    McGowan, Mickie|   25|
|      Proctor, Phil|   24|
|        Cruise, Tom|   23|
|         Pitt, Brad|   23|
|   Wilson, Owen (I)|   23|
|       Depp, Johnny|   22|
|Freeman, Morgan (I)|   22|
|     Morrison, Rana|   22|
|Williams, Robin (I)|   22|
|      Diaz, Cameron|   21|
+-------------------+-----+
only showing top 20 rows



In [9]:
# 2
(
    movies_df
    .groupBy(
        "año"
    )
    .agg(
        countDistinct(col("pelicula"))
    )
    .withColumn(
        "siglo",
        when((col("año") >= 1900) & (col("año") < 2000), lit("XX")).otherwise(lit("XXI"))
    )
    .show(
        truncate=False,
    )
)

+----+---------------+-----+
|año |count(pelicula)|siglo|
+----+---------------+-----+
|1990|11             |XX   |
|1975|1              |XX   |
|1977|3              |XX   |
|2003|76             |XXI  |
|2007|75             |XXI  |
|2006|86             |XXI  |
|1978|3              |XX   |
|1961|1              |XX   |
|1997|66             |XX   |
|1988|9              |XX   |
|1994|16             |XX   |
|1973|1              |XX   |
|1979|5              |XX   |
|2004|86             |XXI  |
|1991|8              |XX   |
|1967|1              |XX   |
|1982|9              |XX   |
|1989|8              |XX   |
|1996|42             |XX   |
|1998|59             |XX   |
+----+---------------+-----+
only showing top 20 rows



In [18]:
# 3
window_spec = Window.partitionBy("año").orderBy(desc("valoracion"))
window_spec_agg = Window.partitionBy("año")

(
    movie_ratings_df
    .withColumn(
        "row_number",
        row_number().over(window_spec).alias("row_number"),
    )
    .withColumn(
        "max_val",
        max("valoracion").over(window_spec_agg).alias("max_val"),
    )
    .filter(
        col("row_number") == 1,
    )
    .select(
        "valoracion",
        "pelicula",
        "max_val",
    )
    .orderBy(
        desc("año"),
    )
    .show()
)

+----------+--------------------+-------+
|valoracion|            pelicula|max_val|
+----------+--------------------+-------+
|      12.5|       The Wolverine|   12.5|
|   12.8205|                1066|12.8205|
|   14.1527|Ang babae sa sept...|14.1527|
|   14.2173|           Beginners|14.2173|
|   13.7234|Kimmy Dora: Kamba...|13.7234|
|   14.0356|         Man on Wire|14.0356|
|   13.7432|     Hostel: Part II|13.7432|
|   13.7696|Love and Other Di...|13.7696|
|   14.1976|             The Man|14.1976|
|   14.2073|           Sleepover|14.2073|
|   14.1829|               Gigli|14.1829|
|   12.8821|         Extreme Ops|12.8821|
|   14.0009|       Tortilla Soup|14.0009|
|   14.1178|              Taxi 2|14.1178|
|   14.0421|       Sofies verden|14.0421|
|    14.015|            My Giant| 14.015|
|   13.7106|  Leave It to Beaver|13.7106|
|      14.1|          Sgt. Bilko|   14.1|
|   13.0879|Something to Talk...|13.0879|
|   14.1242|Änglagård - Andra...|14.1242|
+----------+--------------------+-